## RAG menggunakan Llama2 melalui Ollama

### Setup dan Impor Library

In [1]:
import os
from dotenv import load_dotenv
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain.chains import RetrievalQA

load_dotenv()

True

Pada bagian ini, kita mengimpor semua library yang diperlukan untuk implementasi RAG. Kita menggunakan `langchain_community` untuk akses ke Ollama dan komponen lainnya.

In [2]:
MODEL = "llama2"
model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

Di sini, kita mengatur model Llama2 dan embeddings menggunakan Ollama. Pastikan Anda telah menginstal dan menjalankan Ollama dengan model Llama2 di mesin lokal Anda.

### Membuat Template Prompt

In [3]:
template = """
Answer the question based on the context below. If you can't 
answer the question, reply "Maaf, terkait ini Saya belum dapat memberikan Anda jawaban".

Context: {context}
Question: {question}

Answer:
"""
prompt = PromptTemplate.from_template(template)

Template prompt ini akan digunakan untuk memberi instruksi kepada model bagaimana cara menjawab pertanyaan berdasarkan konteks yang diberikan.

### Memuat dan Memproses Dokumen

In [4]:
loader = PyPDFLoader("./letterC.pdf")
pages = loader.load_and_split()

# Vector store and retriever setup
vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)
retriever = vectorstore.as_retriever()

c:\Users\lenov\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


Pada langkah ini, kita memuat dokumen PDF, membaginya menjadi halaman, dan kemudian membuat vector store dari dokumen tersebut. Retriever digunakan untuk mengambil bagian yang relevan dari dokumen.

### Membuat Chain RetrievalQA

In [5]:
# Create RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=model,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

# Function to ask questions
def ask_question(question):
    result = qa_chain({"query": question})
    return result["result"], result["source_documents"]

RetrievalQA chain menggabungkan proses retrieval dan generation. Ini akan mengambil bagian yang relevan dari dokumen dan menggunakannya untuk menjawab pertanyaan.

### Fungsi untuk Mengajukan Pertanyaan

In [6]:
question = "Apa isi dari letterC?"
answer, sources = ask_question(question)
print(f"Question: {question}")
print(f"Answer: {answer}")
print("\nSources:")
for source in sources:
    print(f"- {source.metadata['page']} : {source.page_content[:100]}...")

C:\Users\lenov\AppData\Local\Temp\ipykernel_25428\297317597.py:12: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result = qa_chain({"query": question})


Question: Apa isi dari letterC?
Answer: The contents of Letter C (Surat Kutipan Letter C Desa) include:

1. Nama dan Nomor Urut Pemilik: The name and serial number of the landowner.
2. Nomor dan Bagian Persil: The lot number and the classification of the land (e.g. Persil 1, Persil 4, etc.).
3. Nama Desa: The name of the village.
4. Tgl: The date.
5. Mengetahui: The signature of the Head of the Village or Kelurahan.
6. Nama Kepala Desa: The name of the Head of the Village or Kelurahan.
7. Fungsi Buku Letter C Desa: The function of the Letter C book in relation to the Land Record Book as a legal document.
8. Daftar Pustaka: A list of reference books, including "Kamus Hukum" by Andi Hamzah, "Majalah Hukum Pro Justitia" by Andreas Lumme, "Pendaftaran Tanah di Indonesia" by AP Perlindungan II, and "Hukum Agraria Nasional" by Boedi Harsono.

The Letter C book is an important document that serves as a proof of ownership of land and can be used as a basis for changes in land ownership due to 

### Catatan Penting

1. Performa: Kecepatan pemrosesan akan bergantung pada spesifikasi hardware, disini kita (hanya) menggunakan CPU Core i5.
2. Penggunaan Memori: Llama2 70B adalah model besar. Operasionalnya membutuhkan RAM yang cukup.
3. Kualitas Output: Hasil mungkin berbeda dari GPT-3.5-turbo (OpenAI API), tergantung pada versi model.
4. Keamanan: Semua pemrosesan dilakukan secara lokal.

### Langkah Selanjutnya

- Minimalisir halusinasi (buku letter c tidak memiliki daftar pustaka)
- Implementasikan caching untuk meningkatkan kinerja.
- Eksperimen dengan ukuran konteks yang berbeda.
- Tambahkan logging untuk monitoring performa.
- Pertimbangkan penggunaan versi Llama2/model lain yang lebih kecil jika mengalami masalah performa atau memori.
- Perbaikan retrieval > generation